# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weather_df = pd.read_csv('../WeatherPy/weatherAPI_data.csv')
weather_df.head()

,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,3833367,Ushuaia,AR,-54.80,-68.30,10/26/20,50.00,31,20,21.92
1,6068416,Mayo,CA,63.59,-135.90,10/26/20,28.40,92,90,5.82
2,2124611,Kholodnyy,RU,62.72,147.95,10/26/20,11.89,85,0,1.79
3,3893629,Coquimbo,CL,-29.95,-71.34,10/26/20,55.40,87,0,10.29
4,4030556,Rikitea,PF,-23.12,-134.97,10/26/20,74.57,86,100,13.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Store latitude and longitude in locations
lat_lon = weather_df[['latitude', 'longitude']]

# Fill NaN values and convert to float
humid = weather_df["humidity"]
lat_lon.head()


,latitude,longitude
0,-54.80,-68.30
1,63.59,-135.90
2,62.72,147.95
3,-29.95,-71.34
4,-23.12,-134.97


In [8]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
crit_temperature = (weather_df.temp < 80) & (weather_df.temp > 70)
crit_windSpeed = weather_df.wind_speed < 15
crit_cloudiness = weather_df.cloudiness == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = weather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,22,3393692,Itarema,BR,-2.92,-39.92,10/26/20,78.55,81,0,13.73
1,139,1174451,Keti Bandar,PK,24.14,67.45,10/26/20,77.90,58,0,8.81
2,222,2359608,Kaya,BF,13.09,-1.08,10/26/20,79.63,17,0,4.99
3,236,286621,Salalah,OM,17.02,54.09,10/26/20,75.20,60,0,4.70
4,290,1168312,Pasni,PK,25.26,63.47,10/26/20,72.84,54,0,8.12
5,311,3388456,São José da Coroa Grande,BR,-8.90,-35.15,10/26/20,77.25,80,0,12.75
6,370,2362344,Bobo-Dioulasso,BF,11.18,-4.30,10/26/20,78.80,69,0,2.24
7,384,1168700,Ormara,PK,25.21,64.64,10/26/20,75.58,55,0,10.47
8,391,1259813,Phalodi,IN,27.13,72.37,10/26/20,71.17,22,0,3.22
9,421,910111,Livingstone,ZM,-17.84,25.85,10/26/20,73.72,31,0,7.31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - I used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
#hotel_df.head()

In [13]:
# Params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Since some data may be missing, incorporate a try-except to skip any missing data.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

#hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [27]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…